In [1]:
suppressPackageStartupMessages({
    library(CellChat)
    library(patchwork)
    library(RhpcBLASctl)
    library(Matrix)
    library(fgsea)
    library(psych)
    library(readxl)
})
options(stringsAsFactors = FALSE)
expression_data_path = '/data2/eric/CCC-Benchmark/data/External/'#'/data2/hratch/immune_CCI/covid/expression_data/'
output_results_path = '/data2/hratch/immune_CCI/covid/results/'
input_data_path = '/data2/hratch/immune_CCI/covid/inputs/'


In [2]:
# filter for just the LR pairs used by Tensor cell2cell
lr_pairs<-read.csv(paste0(input_data_path,'Tensor-cell2cell-LRpairs.csv'))
lr_pairs<-lr_pairs$interaction_name
humandb<-CellChatDB.human
humandb$interaction<-CellChatDB.human$interaction[CellChatDB.human$interaction$interaction_name %in% lr_pairs, ] 

In [ ]:
#' Rank the similarity of the shared signaling pathways based on their joint manifold learning
#'
#' @param object CellChat object
#' @param slot.name the slot name of object that is used to compute centrality measures of signaling networks
#' @param type "functional","structural"
#' @param comparison1 a numerical vector giving the datasets for comparison. This should be the same as `comparison` in `computeNetSimilarityPairwise`
#' @param comparison2 a numerical vector with two elements giving the datasets for comparison.
#'
#' If there are more than 2 datasets defined in `comparison1`, `comparison2` can be defined to indicate which two datasets used for computing the distance.
#' e.g., comparison2 = c(1,3) indicates the first and third datasets defined in `comparison1` will be used for comparison.
#' @import ggplot2
#' @importFrom methods slot
#' @return
#' @export
#'
#' @examples
rankSimilarity_ <- function(object, slot.name = "netP", type = c("functional","structural"), comparison1 = NULL,  
                           comparison2 = c(1,2)) {
  type <- match.arg(type)

  if (is.null(comparison1)) {
    comparison1 <- 1:length(unique(object@meta$datasets))
  }
  comparison.name <- paste(comparison1, collapse = "-")
  cat("Compute the distance of signaling networks between datasets", as.character(comparison1[comparison2]), '\n')
  comparison2.name <- names(methods::slot(object, slot.name))[comparison1[comparison2]]

  Y <- methods::slot(object, slot.name)$similarity[[type]]$dr[[comparison.name]]
  group <- sub(".*--", "", rownames(Y))
  data1 <- Y[group %in% comparison2.name[1], ]
  data2 <- Y[group %in% comparison2.name[2], ]
  rownames(data1) <- sub("--.*", "", rownames(data1))
  rownames(data2) <- sub("--.*", "", rownames(data2))

  pathway.show = as.character(intersect(rownames(data1), rownames(data2)))
  data1 <- data1[pathway.show, ]
  data2 <- data2[pathway.show, ]
  euc.dist <- function(x1, x2) sqrt(sum((x1 - x2) ^ 2))
  dist <- NULL
  for(i in 1:nrow(data1)) dist[i] <- euc.dist(data1[i,],data2[i,])
  df <- data.frame(name = pathway.show, dist = dist, row.names = pathway.show)
  df <- df[order(df$dist), , drop = F]
  df$name <- factor(df$name, levels = as.character(df$name))

  return(df)
}

In [ ]:
#' Run the CellChat pipeline on a single sample from Covid datasets
#'
#' @param data.input <- normalized data matrix (genes x cells) for one sample; dgCMatrix
#' @param meta <- associated metadata for data.input cells; one column must be "celltype"
#' @param organism <- string; default 'human' (only current option)
#' @param n.cores <- number of cores for parallelization (int) default NULL
#' @param score.type <- way to calculate average cell expression, see CellChat::computeCommunProb for details, str, default 'trimean'
#' @param trim <- float [0,1] accompanying score.type, see CellChat::computeCommunProb for details, default NULL
#' @param seed <- int, default NULL
#' @param comparison.type <- string, either functional or 'structural', see CellChat::computeNetSimilarityPairwise, default 'structural'

get_cellchat_sample<-function(data.input, meta, organism = 'human', n.cores = NULL, score.type = 'triMean', 
                              trim = NULL, seed = NULL){
    
    # loop through each context and create a cell type future
    cellchat <- createCellChat(object = data.input, meta = meta, group.by = 'cell_type')
    
    if (organism == 'human'){
        cellchat@DB <- humandb # human organism
    }else{stop('Only human organism considered currently')}
    # parallelization
    if (!(is.null(n.cores)) | (!n.cores<=1)){
        RhpcBLASctl::blas_set_num_threads(n.cores)
        future::plan("multiprocess", workers = n.cores) 
    }else{
        RhpcBLASctl::blas_set_num_threads(1)
    }
    cellchat <- subsetData(cellchat) # subset the expression data of signaling genes, assign to @data.signalling 


    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat) # generate @ LR slot used by computeCommunProb
    cellchat <- projectData(cellchat, PPI.human) # shallow sequencing depth
    
    # raw.use = F assumes shallow sequencing depth (10x used on COVID dataset)
    # population.size = T considers effect of cell proportion; since a CDR correction is applied, we set to F
    cellchat <- computeCommunProb(cellchat, raw.use = F, type = score.type, trim = trim, seed.use = seed, 
                                 population.size = F) 
    
    # The functional similarity analysis requires the same cell population composition between two datasets.
    cellchat <- filterCommunication(cellchat, min.cells = 10)
    cellchat <- computeCommunProbPathway(cellchat)
    return(cellchat)
    }

#' Run the CellChat pipeline on multiple samples (contexts across Covid datasets)
#'
#' @param object.list <- list of cellchat objects, each for a different sample, output from get_cellchat_sample
#' @param n.cores <- number of cores for parallelization (int) default NULL
#' @param comparison.type <- string, either functional or 'structural', see CellChat::computeNetSimilarityPairwise, default 'structural'

get_cellchat_context<-function(object.list, n.cores = NULL, comparison.type = 'structural'){
    cellchat <- mergeCellChat(object.list, add.names = names(object.list))
    cellchat <- computeNetSimilarityPairwise(cellchat, type = comparison.type)
    cellchat <- netEmbedding(cellchat, type = comparison.type)
    # Manifold learning of the signaling networks for datasets 
    if (!(is.null(n.cores)) | (!n.cores<=1)){
        cellchat <- netClustering(cellchat, type = comparison.type, nCores = n.cores, do.plot = F)
    }else{
        cellchat <- netClustering(cellchat, type = comparison.type, do.parallel = F, do.plot = F)
    }
#     path.dist<-rankSimilarity_(cellchat, type = comparison.type) #CHANGE to pairwise comparison
    pairwise_comparisons<-sapply(as.data.frame(combn(seq(1:length(object.list)),2)), 
                             function(x) as.numeric(x), simplify = F) # pairwise combination of elements
    path_dis_res = list()
    for (pc in names(pairwise_comparisons)){
        print(pc)
        path.dist<-rankSimilarity_(cellchat, type = comparison.type, comparison1 = 1:length(object.list),
                                   comparison2 = pairwise_comparisons[[pc]]) 
        path_dis_res[[paste(names(object.list)[pairwise_comparisons[[pc]]], collapse = '_')]]<-path.dist
    }                            
#     ifl<-rankNet(cellchat, mode = 'comparison', do.stat = T, return.data = T) 
                                 
    if_res = list()
    for (pc in names(pairwise_comparisons)){
        ifl<-rankNet(cellchat, mode = 'comparison', do.stat = T, return.data = T, 
                    comparison = pairwise_comparisons[[pc]]) 
        if_res[[paste(names(object.list)[pairwise_comparisons[[pc]]], collapse = '_')]]<-ifl$signaling.contribution
    }
    res = list()
    res[['pathway_distance']] = path_dis_res
    res[['information.flow']] = if_res
    return(res)
}

# cellchat on individual samples

In [3]:
organism = 'human'
n.cores = 17
score.type = 'triMean'
trim = NULL
comparison.type = 'structural'
# if by sample, will create a separate cellchat object for each sample, otherwise, will first merge by 
# the condition, and create a separate cellchat object for each condition
by.sample<-FALSE 
if (by.sample){seed<-24}else{seed<-25}

In [ ]:
# load input data
fns = list()
for (fn in list.files(expression_data_path)){
    sn = strsplit(fn, '_')[[1]]
    sample.name = sn[[2]]
    type = sn[[1]]
    fns[[sample.name]][[type]] = fn
}

In [ ]:
# create cellchat object for each sample or context
covid.list<-list()
if (!by.sample){ # by context
    # merge the metadata files
    print('Read in metadata files')
    meta_all <- read.csv(paste0(expression_data_path, fns[[1]]$Meta))
    for (sample.name in names(fns)[2:length(fns)]){
        meta_ <- read.csv(paste0(expression_data_path, fns[[sample.name]]$Meta))
        meta_all<-rbind(meta_all,meta_)
    }
    
    if (length(unique(meta_all$Cell)) != dim(meta_all)[[1]]){
        stop('Overlapping cell barcodes across samples')
    }
    
    # map samples to context
    context_map<-meta_all$sample
    names(context_map)<-meta_all$group
    context_map<-context_map[!duplicated(context_map)]
    context_map<-setNames(names(context_map), context_map)
    contexts = unique(context_map)
    
    # separate metadata by context
    meta_map<-list()
    for (context in contexts){
        meta_map[[context]]<-meta_all[meta_all$group==context,]
    }
    
    # create context-merged expression matrices
    data_input_map<-rep(list(NULL), each = 3)
    names(data_input_map)<-contexts
    counter<-1
    print('Merge expression matrices by context: ')
    for (sample.name in names(fns)){
        print(paste0(counter, ' of ', length(fns)))
        context = context_map[[sample.name]]
        sample.data<-read.csv(paste0(expression_data_path, fns[[sample.name]]$DGE))
        rownames(sample.data)<-sample.data$Gene
        sample.data<-sample.data[ , !(colnames(sample.data) %in% c('Gene'))]
        if (is.null(data_input_map[[context]])){ 
            data_input_map[[context]]<-sample.data
        }
        else{
            if (dim(sample.data)[[1]] != dim(data_input_map[[context]])[[1]]){ # genes are same in all dfs so don't have to worry about this
                stop('Not the same genes')
            }else{sample.data<-sample.data[rownames(data_input_map[[context]]),]}
            data_input_map[[context]]<-cbind(data_input_map[[context]], sample.data)
        }
        counter<-counter + 1
        print('------------------------------')    
    }
    
    # create context-specific cellchat objects
    print('Create context-specific cellchat objects')
    counter<-1
    for (context in contexts){
        print(paste0(counter, ' of ', length(contexts)))
        data.input<-data_input_map[[context]]
        meta<-meta_map[[context]]

        cell.names<-colnames(data.input)
        gene.names<-rownames(data.input)
        print('Convert to sparse dgcmatrix')
        data.input<-Reduce(cbind2, lapply(data.input, Matrix, sparse = TRUE))# convert to sparse dgcmatrix, slow
        colnames(data.input) = cell.names
        rownames(data.input) = gene.names

        rownames(meta) = meta$Cell
        meta = meta[colnames(data.input), ] # order

        # generate the cellchat object
        cellchat.obj<-get_cellchat_sample(data.input = data.input, meta = meta, organism = organism, 
                                      n.cores = n.cores, score.type = score.type, 
                                  trim = trim, seed = seed)
        covid.list[[context]]<-cellchat.obj
        counter<-counter + 1
        print('------------------------------')
    }
    saveRDS(covid.list, paste0(output_results_path, 'cellchat_list_bycontext.rds')) # checkpoint
}else{ # by sample
  counter<-1
    for (sample.name in names(fns)){
        print(paste0(counter, ' of ', length(fns)))
        print('Read in data')
        data.input<-read.csv(paste0(expression_data_path, fns[[sample.name]]$DGE))
        rownames(data.input)<-data.input$Gene
        data.input<-data.input[ , !(colnames(data.input) %in% c('Gene'))]
        # # SUBSETTING, REMOVE
        # data.input<-data.input[sample(rownames(data.input), size = round(dim(data.input)[[1]]*0.3)), 
        #            sample(colnames(data.input), size = round(dim(data.input)[[2]]*0.3))]

        cell.names<-colnames(data.input)
        gene.names<-rownames(data.input)
        data.input<-Reduce(cbind2, lapply(data.input, Matrix, sparse = TRUE))# convert to sparse dgcmatrix, slow
        colnames(data.input) = cell.names
        rownames(data.input) = gene.names

        meta = read.csv(paste0(expression_data_path, fns[[sample.name]]$Meta))
        rownames(meta) = meta$Cell
        meta = meta[colnames(data.input), ] # order

        # generate the cellchat object
        print('Generate cellchat object')
        cellchat.obj<-get_cellchat_sample(data.input = data.input, meta = meta, organism = organism, 
                                      n.cores = n.cores, score.type = score.type, 
                                  trim = trim, seed = seed)
        covid.list[[sample.name]]<-cellchat.obj

        counter<-counter + 1
        print('------------------------------')
        }
    saveRDS(covid.list, paste0(output_results_path, 'cellchat_list_bysample.rds')) # checkpoint  
}

# merged cellchat

In [ ]:
# compare across contexts
if (by.sample){
    covid.list<-readRDS(paste0(output_results_path, 'cellchat_list_bysample.rds')) 
    res<-get_cellchat_context(object.list = covid.list, n.cores = length(covid.list), comparison.type = 'structural')
    saveRDS(res, paste0(output_results_path, 'cellchat_context_results_bysample.rds')) # checkpoint
}else{
    covid.list<-readRDS(paste0(output_results_path, 'cellchat_list_bycontext.rds')) 
    res<-get_cellchat_context(object.list = covid.list, n.cores = length(covid.list), comparison.type = 'structural')
    saveRDS(res, paste0(output_results_path, 'cellchat_context_results_bycontext.rds')) # checkpoint
}

In [ ]:
# save results as csvs
res<-readRDS(paste0(output_results_path, 'cellchat_context_results_bycontext.rds')) 
for (comparison.type in names(res)){
    for (comparison.pair in names(res[[comparison.type]])){
        write.csv(res[[comparison.type]][[comparison.pair]], 
                 paste0(output_results_path, 'cellchat_', comparison.type, '_', comparison.pair, '.csv'))
    }
}

# Compare Cellchat and Tensor-Cell2Cell 

In [4]:
n.cores = 17
fn = 'Enrichment_LR_Loadings.csv'

In [5]:
loadings = read.csv(paste0(input_data_path, fn))
cellchat.res<-readRDS(paste0(output_results_path, 'cellchat_context_results_bycontext.rds')) 

In [6]:
# format inputs
term_to_gene = data.frame(matrix(ncol = 2, nrow = dim(loadings)[[1]]))
term_to_gene[[1]] = loadings$Pathways
term_to_gene[[2]] = loadings$X
colnames(term_to_gene) = c('Signalling.Pathway', 'Ligand.Receptor.Pair')
rownames(loadings) = loadings$X
loadings<-loadings[, !(colnames(loadings) %in% c('X', 'Pathways'))]

### Enrichment Analysis

In [7]:
sps <- unique(term_to_gene$Signalling.Pathway)
pathways<-lapply(sps, function(x) term_to_gene[term_to_gene$Signalling.Pathway == x, 'Ligand.Receptor.Pair'])
names(pathways) = sps

In [8]:
# do the enrichment
minGSsize = 3
enrich.res<-NULL
ora.res<-NULL
for (factor in colnames(loadings)){
    factor.list<-loadings[[factor]]
    names(factor.list)<-rownames(loadings)
    factor.list<-sort(factor.list, decreasing = FALSE)
#     factor.enrich.res<-fgseaSimple(pathways = pathways, stats = factor.list, nperm = 100000, minSize = 1, 
#                scoreType = 'pos', nproc = n.cores, gseaParam = 1)
    factor.enrich.res<-fgseaMultilevel(pathways = pathways, stats = factor.list, nPermSimpl = 100000, 
                                         nproc = n.cores, scoreType = 'pos', minSize = minGSsize)
    factor.ora.res<-fora(pathways = pathways, genes = names(tail(factor.list, 50)), 
                         universe = term_to_gene$Ligand.Receptor.Pair, minSize = minGSsize)

    factor.enrich.res[['Factor']]<-factor
    factor.ora.res[['Factor']]<-factor
    if (!is.null(enrich.res)){
        enrich.res<-rbind(enrich.res, factor.enrich.res)
    }else{enrich.res<-factor.enrich.res}
    
    if (!is.null(ora.res)){
        ora.res<-rbind(ora.res, factor.ora.res)
    }else{ora.res<-factor.ora.res}
}

In [143]:
ora.res$overlapGenes<-vapply(ora.res$overlapGenes, paste, collapse = ", ", character(1L))
write.csv(ora.res, paste0(output_results_path, 'ora_cellchat.csv'))

for (col in c('overlapGenes', 'leadingEdge')){
    enrich.res[[col]]<-vapply(enrich.res[[col]], paste, collapse = ", ", character(1L))
}
write.csv(enrich.res, paste0(output_results_path, 'enrich_cellchat.csv'))

### Jaccard
Get jaccard between ORA results from tensor loadings and cellchat pathway distances

In [11]:
jaccard<-function(list1, list2){
    denom<-length(union(list1,list2))
    if (denom != 0){
        return(length(intersect(list1,list2))/denom)
    }
    else{return(null)}
    
}

In [12]:
 # n = 4 pathways to look at in cellchat, as this is the maximum number of pathwyas for a single 
# factor from tensor ORA
# n_paths<-max(table(ora.res[(ora.res$padj < 0.1),'Factor']))
n_paths<-15

jaccard.compare.ora<-data.frame(matrix(ncol = 3, nrow = length(cellchat.res$pathway_distance)*length(unique(ora.res$Factor))))
colnames(jaccard.compare.ora)<-c('CellChat.comparison', 'Tensor.Factor', 'Jaccard')

counter<-1
for (comparison in names(cellchat.res$pathway_distance)){
    for (factor in unique(ora.res$Factor)){
        pd<-cellchat.res$pathway_distance[[comparison]]
        cellchat_paths<-pd[with(pd, order(-dist)), 'name'][1:n_paths] # filter for top n = 4 pathways by distance
        
        tensor_paths<-ora.res[(ora.res$Factor == factor) & (ora.res$padj < 0.1),][['pathway']]
        
        js<-jaccard(cellchat_paths, tensor_paths)
        jaccard.compare.ora[counter, ]<-c(comparison, factor, js)
        counter<-counter + 1
        
    }
}
write.csv(jaccard.compare.ora, paste0(output_results_path, 'jaccard_ora_cellchat_tc2c.csv'))

 # n = 4 pathways to look at in cellchat, as this is the maximum number of pathwyas for a single 
# factor from tensor enrich
# n_paths<-max(table(enrich.res[(enrich.res$padj < 0.1),'Factor']))
n_paths<-15

jaccard.compare.enrich<-data.frame(matrix(ncol = 3, nrow = length(cellchat.res$pathway_distance)*length(unique(enrich.res$Factor))))
colnames(jaccard.compare.enrich)<-c('CellChat.comparison', 'Tensor.Factor', 'Jaccard')

counter<-1
for (comparison in names(cellchat.res$pathway_distance)){
    for (factor in unique(enrich.res$Factor)){
        pd<-cellchat.res$pathway_distance[[comparison]]
        cellchat_paths<-pd[with(pd, order(-dist)), 'name'][1:n_paths] # filter for top n = 4 pathways by distance
        
        tensor_paths<-enrich.res[(enrich.res$Factor == factor) & (enrich.res$padj < 0.1),][['pathway']]
        
        js<-jaccard(cellchat_paths, tensor_paths)
        jaccard.compare.enrich[counter, ]<-c(comparison, factor, js)
        counter<-counter + 1
        
    }
}
write.csv(jaccard.compare.enrich, paste0(output_results_path, 'jaccard_enrich_cellchat_tc2c.csv'))

### Geometric Mean

geometric mean of loadings within signalling pathway from tensor-cell2cell run at L-R resolution, compared to "pathway distance" of cellchat output

In [14]:
nlps<-c(0,1,2)
# extract the cellchat result to work with (pathway distances)
pds<-cellchat.res$pathway_distance

# initialize df
corr.res.all<-data.frame(matrix(ncol = dim(loadings)[[2]] + 3, nrow=0))
colnames(corr.res.all)<-c(colnames(loadings),c('Cellchat.Comparison', 'Min.GeneSet.Size', 'Correlation.Type'))

for (n_lr_pairs in nlps){
    for (ct in c('pearson', 'spearman')){
        # initialize df
        corr.res<-data.frame(matrix(ncol = dim(loadings)[[2]] + 3, nrow = length(pds)))
        colnames(corr.res)<-colnames(corr.res.all)
        corr.res[['Cellchat.Comparison']]<-names(pds)
        corr.res[['Min.GeneSet.Size']]<-n_lr_pairs+1
        corr.res[['Correlation.Type']]<-ct

        # filter out signalling pathways with only 1 LR pair
        if (n_lr_pairs<0){
            filter<-NULL
        }else{
            filter<-names(which(table(term_to_gene$Signalling.Pathway) == n_lr_pairs))
            filter<-which(names(pathways) %in% filter)
        }

        for (factor in colnames(loadings)){
            # get geometric mean of loadings for each signalling pathway, within the factor
            gms<-as.numeric(lapply(names(pathways), function(sp) psych::geometric.mean(loadings[pathways[[sp]], factor])))
            gms[filter]<-NA
            

            # get pairwise pearson correlation between geometric mean and pathway distance for each cellchat comparison
            corr.res[[factor]]<-lapply(names(pds), function(pd) cor(cbind(gms, pds[[pd]][names(pathways), 'dist'], deparse.level = 0), 
                               use = 'pairwise.complete.obs', method = ct)[1,2])
        }
        corr.res.all<-rbind(corr.res.all, corr.res)
    }
}

In [15]:
for (col.name in colnames(corr.res.all)[1:10]){
    corr.res.all[[col.name]]<-as.numeric(corr.res.all[[col.name]])
}
write.csv(corr.res.all, paste0(output_results_path, 'correlation_cellchat_tc2c.csv'))

In [ ]:
for (fn in c(paste0(output_results_path, 'correlation_cellchat_tc2c.csv'),
paste0(output_results_path, 'jaccard_cellchat_tc2c.csv'),
paste0(output_results_path, 'ora_cellchat.csv'))){
    print(fn)
}

In [26]:
df<-corr.res.all[c('Factor.1', 'Factor.2', 'Factor.3', 'Factor.8', 'Factor.9', 'Cellchat.Comparison', 'Min.GeneSet.Size', 
              'Correlation.Type')]
df[(df$Min.GeneSet.Size == 3) & (df$Correlation.Type == 'spearman'), ]

,Factor.1,Factor.2,Factor.3,Factor.8,Factor.9,Cellchat.Comparison,Min.GeneSet.Size,Correlation.Type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
16,0.05324675,0.47272727,-0.16753247,-0.07402597,-0.3831169,HC_M,3,spearman
17,0.39090909,0.34415584,-0.03636364,-0.05194805,-0.2727273,HC_S,3,spearman
18,0.15714286,-0.06551724,0.15073892,-0.06502463,-0.0320197,M_S,3,spearman


### Tensor-cell2cell pathway level

In [21]:
loadings<-readxl::read_xlsx(paste0(input_data_path, 'COVID-19-Loadings.xlsx'), sheet = 2)
rnames<-loadings[[colnames(loadings)[[1]]]]
loadings<-loadings[,2:dim(loadings)[2]]
rownames(loadings)<-rnames

cellchat.res<-readRDS(paste0(output_results_path, 'cellchat_context_results_bycontext.rds')) 
pd<-cellchat.res$pathway_distance

New names:
* `` -> ...1

Warning message:
“Setting row names on a tibble is deprecated.”


In [78]:
ct<-'spearman'
cor.res<-lapply(names(pd), function(n) (
                   cor(cbind(loadings[rownames(pd[[n]]),], pd[[n]][['dist']]), 
                       method = ct, use = 'pairwise.complete.obs')[dim(loadings)[2] + 1, 1:dim(loadings)[2]]))
names(cor.res)<-names(pd)
cor.res

$HC_M
    Factor 1     Factor 2     Factor 3     Factor 4     Factor 5     Factor 6 
-0.158064516  0.050403226 -0.009677419 -0.183064516 -0.049596774 -0.337096774 
    Factor 7     Factor 8     Factor 9    Factor 10 
-0.424596774 -0.391935484 -0.262500000  0.073387097 

$HC_S
    Factor 1     Factor 2     Factor 3     Factor 4     Factor 5     Factor 6 
-0.096551724  0.249261084 -0.108374384 -0.183251232  0.008866995 -0.301970443 
    Factor 7     Factor 8     Factor 9    Factor 10 
-0.337931034 -0.136453202 -0.173891626 -0.050738916 

$M_S
    Factor 1     Factor 2     Factor 3     Factor 4     Factor 5     Factor 6 
 0.109930314 -0.009407666  0.184146341 -0.070383275  0.201393728 -0.266027875 
    Factor 7     Factor 8     Factor 9    Factor 10 
 0.036933798  0.076829268  0.071951220 -0.139721254

In [79]:
ct<-'pearson'
cor.res<-lapply(names(pd), function(n) (
                   cor(cbind(loadings[rownames(pd[[n]]),], pd[[n]][['dist']]), 
                       method = ct, use = 'pairwise.complete.obs')[dim(loadings)[2] + 1, 1:dim(loadings)[2]]))
names(cor.res)<-names(pd)
cor.res

$HC_M
    Factor 1     Factor 2     Factor 3     Factor 4     Factor 5     Factor 6 
-0.093441074 -0.047686604 -0.145485628 -0.301785463 -0.104959901 -0.255004184 
    Factor 7     Factor 8     Factor 9    Factor 10 
-0.328188508 -0.410319464 -0.288064037  0.006059354 

$HC_S
   Factor 1    Factor 2    Factor 3    Factor 4    Factor 5    Factor 6 
-0.05164586  0.20174811 -0.18674097 -0.23180227 -0.08532119 -0.22082353 
   Factor 7    Factor 8    Factor 9   Factor 10 
-0.26474083 -0.13255580 -0.27033208 -0.07000520 

$M_S
   Factor 1    Factor 2    Factor 3    Factor 4    Factor 5    Factor 6 
 0.14516614 -0.16076230 -0.11682692 -0.16710382  0.14083221 -0.11252228 
   Factor 7    Factor 8    Factor 9   Factor 10 
-0.11623773 -0.04158271  0.02152180 -0.19029793

In [22]:
# get pairwise pearson correlation between geometric mean and pathway distance for each cellchat comparison
corr.res[[factor]]<-lapply(names(pds), function(pd) cor(cbind(gms, pds[[pd]][names(pathways), 'dist'], deparse.level = 0), 
                   use = 'pairwise.complete.obs', method = ct)[1,2])

In [23]:
for (n_lr_pairs in nlps){
    for (ct in c('pearson', 'spearman')){
        # initialize df
        corr.res<-data.frame(matrix(ncol = dim(loadings)[[2]] + 3, nrow = length(pds)))
        colnames(corr.res)<-colnames(corr.res.all)
        corr.res[['Cellchat.Comparison']]<-names(pds)
        corr.res[['Min.GeneSet.Size']]<-n_lr_pairs+1
        corr.res[['Correlation.Type']]<-ct

        # filter out signalling pathways with only 1 LR pair
        if (n_lr_pairs<0){
            filter<-NULL
        }else{
            filter<-names(which(table(term_to_gene$Signalling.Pathway) == n_lr_pairs))
            filter<-which(names(pathways) %in% filter)
        }

        for (factor in colnames(loadings)){
            # get geometric mean of loadings for each signalling pathway, within the factor
            gms<-as.numeric(lapply(names(pathways), function(sp) psych::geometric.mean(loadings[pathways[[sp]], factor])))
            gms[filter]<-NA
            

            # get pairwise pearson correlation between geometric mean and pathway distance for each cellchat comparison
            corr.res[[factor]]<-lapply(names(pds), function(pd) cor(cbind(gms, pds[[pd]][names(pathways), 'dist'], deparse.level = 0), 
                               use = 'pairwise.complete.obs', method = ct)[1,2])
        }
        corr.res.all<-rbind(corr.res.all, corr.res)
    }
}

ERROR: Error in rbind(deparse.level, ...): numbers of columns of arguments do not match
